In [1]:
from imports import *
from data_generator import *
from models import *

#shouldn't need on a local computer?
from utils import limit_mem

#Module to save and load models
import h5py
import netCDF4
import numpy as np

from keras.models import Sequential
from keras.layers import *
import time
#added by Griffin
import xarray as xr


from tensorflow.python.ops.parallel_for.gradients import batch_jacobian

Using TensorFlow backend.
/export/home/gmooers/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/export/home/gmooers/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/export/home/gmooers/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [2]:
def custom_activation(z):
    #y = x**10*K
    #y = K.exp(z)
    y = z**10
    return y

In [3]:
output_vector = 65
input_vector = 64
print('Starting')


#If running on the GPU or GPU-shared partition uncomment
limit_mem()


DATADIR = '/DFS-L/DATA/pritchard/gmooers/RG_Paper/RG_DATA/Preprocessed_Data/One_Month_July/'



valid_gen = DataGenerator(
    data_dir=DATADIR, 
    feature_fn='full_physics_essentials_valid_month02_features.nc',
    target_fn='full_physics_essentials_valid_month02_targets.nc',
    batch_size=512,
    norm_fn='full_physics_essentials_train_month01_norm.nc',  # SAME NORMALIZATION FILE!
    fsub='feature_means', 
    fdiv='feature_stds', 
    tmult='target_conv',
    shuffle=False,
)


fsub = valid_gen.feature_norms[0]
fdiv = valid_gen.feature_norms[1]
tsub = valid_gen.target_norms[0]
tdiv = valid_gen.target_norms[1]


model = tf.keras.models.load_model('/fast/gmooers/Models/ContinentModels/second_round_model.h5')

path_to_file = '/DFS-L/DATA/pritchard/gmooers/RG_Paper/RG_DATA/Preprocessed_Data/One_Month_July/full_physics_essentials_valid_month02_features.nc'
real_ds = xr.open_dataset(path_to_file)
features = real_ds.features[:, :].values
print(features.shape)


path_to_file = '/DFS-L/DATA/pritchard/gmooers/RG_Paper/RG_DATA/Preprocessed_Data/One_Month_July/full_physics_essentials_valid_month02_targets.nc'
real_ds = xr.open_dataset(path_to_file)
targets = real_ds.targets[:, :].values
print('files imported')
model_data = np.zeros(shape=(len(features), output_vector))
model_data[:,:] = np.nan

W1110 15:22:45.838151 140272943224640 deprecation_wrapper.py:119] From /fast/gmooers/Real_Geography_Manuscript/utils.py:237: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W1110 15:22:45.840032 140272943224640 deprecation_wrapper.py:119] From /fast/gmooers/Real_Geography_Manuscript/utils.py:239: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



Starting


W1110 15:22:49.757701 140272943224640 deprecation.py:506] From /export/home/gmooers/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1110 15:22:49.759760 140272943224640 deprecation.py:506] From /export/home/gmooers/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1110 15:22:49.762934 140272943224640 deprecation.py:506] From /export/home/gmooers/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/o

Generator will have 41126400 samples in 80325 batches
Features have shape 64; targets have shape 65
(41126400, 64)
files imported


In [4]:
def get_jacobian(x, model):
    sess = tf.keras.backend.get_session()
    jac = jacobian(model.output, model.input)
    J = sess.run(jac, feed_dict={model.input: x.astype(np.float32)[None]})
    return J.squeeze()
def get_batch_jacobian(x, model):
    sess = tf.keras.backend.get_session()
    jac = batch_jacobian(model.output, model.input)
    J = sess.run(jac, feed_dict={model.input: x.astype(np.float32)})
    return J.squeeze()

In [5]:
def get_jacobian(model,inp,sample_index,gen_obj,fdiv,cf):
# model is the neural network model from inp to out
# inp is the input x generator from the generator (object = gen_obj)
# sample_index is the reference number of the sample for x
# x.shape = (#sample,#inputs) so we are evaluating the gradient of
# y(sample_index,:) with respect to x(sample_index,:)
    J = np.zeros((gen_obj.target_shape, gen_obj.feature_shape))
    for i in range(gen_obj.target_shape):
        with tf.GradientTape() as tape:
            x_tf = tf.convert_to_tensor(np.expand_dims(x[sample_index,:],axis=0) )
            tape.watch(x_tf)
            pred = model(x_tf)
            out_i = pred[0, i]
        grads = tape.gradient(out_i, x_tf).numpy()
        J[i, :] = grads/(fdiv*cf)
    return J

In [100]:
x = tf.constant([[4, 2],[1, 3]], dtype=tf.dtypes.float32) 
with tf.GradientTape() as gg: 
    gg.watch(x) 
    y = x * x * x 
   # Computing first order jacobian 
first_order = gg.jacobian(y, x)
print(first_order)

Tensor("Reshape_433:0", shape=(2, 2, 2, 2), dtype=float32)


In [134]:
J = np.zeros((65, 64))
#x_tf = tf.convert_to_tensor(np.expand_dims(features[0,:],axis=0))
#f = (x_tf-fsub)/fdiv
with tf.GradientTape() as tape:
    #x_tf = tf.convert_to_tensor(np.expand_dims(features[0,:],axis=0))
    x_tf = tf.convert_to_tensor(np.expand_dims(features[0,:],axis=0))
    tape.watch(x_tf)
    f = (x_tf-fsub)/fdiv
   # p = tf.convert_to_tensor(model.predict_on_batch(f))
    p = f*f
    #print(f.shape)
    #pred = (p/tdiv)+tsub
    print(type(p))
    print(p.shape)
    #pred = tf.convert_to_tensor(pred)
    #print(type(pred))
    #out_i = pred[0, 0]
    #print(out_i*10000,0)

#grads = tape.jacobian(p, f)
print(grads)
#J[i, :] = grads/(fdiv*cf)

<class 'tensorflow.python.framework.ops.Tensor'>
(1, 64)
Tensor("Reshape_464:0", shape=(1, 64, 1, 64), dtype=float32)


In [133]:
dir(model)

['_TF_MODULE_IGNORED_PROPERTIES',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activity_regularizer',
 '_add_inbound_node',
 '_add_unique_metric_name',
 '_add_variable_with_custom_getter',
 '_all_metrics_tensors',
 '_assert_compile_was_called',
 '_assert_weights_created',
 '_base_init',
 '_cache_output_metric_attributes',
 '_call_convention',
 '_call_fn_args',
 '_call_metric_fn',
 '_callable_losses',
 '_check_trainable_weights_consistency',
 '_checkpoint_dependencies',
 '_ckpt_saved_epoch',
 '_clear_losses',
 '_cloning',
 '_collected_trainable_weights',
 '_compile_distribution',
 '_compile_eagerly',
 '_compile_metric_functions',
 '_compile_metrics',
 

In [115]:
model.output(f)

TypeError: 'Tensor' object is not callable

In [69]:
for i in range(96):
    with tf.GradientTape() as tape:
        x_tf = tf.convert_to_tensor(np.expand_dims(features[i,:],axis=0))
        print(type(x_tf))
        tape.watch(x_tf)
        f = (x_tf-fsub)/fdiv
        print(type(f))
        p = model.predict_on_batch(f)
        print(type(p))
        p = (p/tdiv)+tsub
        print(type(p), p.shape)
        print(x_tf.shape)
        pred = tf.convert_to_tensor(p)
        print(type(pred))
        out_i = pred[0, i]
      
        
        
    #tf.print(tape.jacobian(pred, x_tf))
    #print(tape.gra,dient(pred, x_tf))
    grads = tape.jacobian(out_i, x_tf).numpy()
    J[i, :] = grads/(fdiv*cf)
    

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'> (1, 65)
(1, 64)
<class 'tensorflow.python.framework.ops.Tensor'>


AttributeError: 'NoneType' object has no attribute 'numpy'

In [60]:
from tensorflow.python.ops.parallel_for.gradients import jacobian

In [65]:
for i in range(96):
    f = features[i]
    f=f.reshape(-1,1)
    x = np.reshape(f, (1,64))
    f = (x_tf-fsub)/fdiv
    p = model.predict_on_batch(f)
    p = (p/tdiv)+tsub
    
    J=jacobian(tf.convert_to_tensor(p),tf.convert_to_tensor(x))
    if i == 0:
        print(p.shape)
        print(x.shape)
        print(J)
        print(J)


(1, 65)
(1, 64)
None
None


In [ ]:
from tensorflow.python.ops.parallel_for.gradients import jacobian
J=jacobian(Y,X)

In [66]:
def jacobian_tensorflow(x):    
    jacobian_matrix = []
    for m in range(Neuron_Out):
        # We iterate over the M elements of the output vector
        grad_func = tf.gradients(model_ANN.output[:, m],model_ANN.input)
        gradients = sess.run(grad_func, feed_dict={model_ANN.input: x})  # Removed x.reshape((1, x.size)) as reshape is not required bcoz dense accepts the shape
        jacobian_matrix.append(gradients[0][0,:])

    return np.array(jacobian_matrix)

In [ ]:
for i in range(96):
    f = features[i]
    f=f.reshape(-1,1)
    x = np.reshape(f, (1,64))
    f = (x_tf-fsub)/fdiv
    p = model.predict_on_batch(f)
    p = (p/tdiv)+tsub
    
    J=jacobian(tf.convert_to_tensor(p),tf.convert_to_tensor(x))
    if i == 0:
        print(p.shape)
        print(x.shape)
        print(J)
        print(J)


In [7]:
#gradients = sess.run(grad_func, feed_dict={model.input: x.reshape((1, x.size))})
#        jacobian_matrix.append(gradients[0][0,:])